<a href="https://colab.research.google.com/github/ssundar6087/Deep-Learning-Mini-Course/blob/main/Keras/DL_Minicourse_Keras_Day_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Deconstructing the training loop
Today, we'll look at the training and validation loops and play with hyperparameters. Note that 90% of the code will be the same as the previous notebook. Our focus is to play with the hyperparameters and see if we can improve results.

# Image Classification Keras

In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

## Load Dataset - CIFAR-10
The dataset has 10 classes with 50k training images and 10k test images

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = x_train / 255.0, x_test / 255.0

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Show Some Images

In [ ]:
plt.figure(figsize=(8,8))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(classes[y_train[i][0]])
plt.show()

## Define our Model

In [ ]:
def get_baby_thanos(input_shape, num_classes):
  inputs = keras.Input(shape=input_shape)
  x = keras.layers.Dense(256, activation='relu')(inputs)
  x = keras.layers.Dense(128, activation='relu')(x)
  outputs = keras.layers.Dense(num_classes)(x)
  return keras.Model(inputs, outputs)    

In [ ]:
IN_FTRS = 32 * 32 * 3 # CIFAR-10 images are 32 x 32 and have 3 channels
net = get_baby_thanos(input_shape=IN_FTRS, num_classes=10)

In [ ]:
net.summary()

## Flatten Images

In [ ]:
train_images = train_images.reshape(-1, IN_FTRS)
test_images = test_images.reshape(-1, IN_FTRS)

In [ ]:
print(train_images.shape, test_images.shape)

## **YOUR EXERCISE HERE: Hyperparameters to tune** 👇
While there are a ton of hyperparameters that we can play with, we'll focus on the three most commonly tuned ones:
- Epochs : Number of times the model sees the entire dataset
- Batch Size: How many examples of the dataset the model sees at a time
- Learning Rate: How much the gradients affect the weights (See the optimizer call below)

Change these values one at a time and observe how the training curves change. As a suggestion, play with the learning rate first and then try changing the batch size and then the number of epochs. How does the model's performances change?

In [ ]:
LEARNING_RATE = 0.001
EPOCHS = 20
BATCH_SIZE = 64

## Define Optimizer & Loss Function
These two functions allow us to help baby thanos learn from his mistakes.

In [ ]:
criterion = keras.losses.SparseCategoricalCrossentropy(from_logits=True) # Loss Function
optimizer = keras.optimizers.SGD(learning_rate=LEARNING_RATE) # Optimizer 

## Train and Evaluate the Network
The training and validation loops call the same set of functions over and over. Keras packages them into a nice function called `fit()` which does a lot more :)

In [ ]:
net.compile(
    optimizer=optimizer,
    loss=criterion,
    metrics=["accuracy"],
)

In [ ]:
history = net.fit(train_images, 
                  y_train, 
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS, 
                  validation_data=(test_images, y_test),
                  )

## Plot the Loss and Accuracy of our Model

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.grid("on")
plt.legend()
plt.title("Loss vs Epochs");

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.grid("on")
plt.legend()
plt.title("Accuracy vs Epochs");